In [69]:
import pandas as pd
from collections import Counter
import math

In [21]:
teaming = pd.read_csv('../results/20170722-112357/teaming2.out', header=None, names=['hash','team','semester'])
data = pd.read_csv('../project4.csv')

In [22]:
teaming

,hash,team,semester
0,642bddd5d6f11393f0a2c224e0cb6ad6,0,WT-15
1,04074a0671bb5b51850d6678b51b4497,0,WT-15
2,2e82abf8e85b5ff209870fe67da5746e,0,WT-15
3,d0c03244272fc074a85681a697760aa8,0,WT-15
4,40a526979ae2704d2abd2d4a57df330b,0,WT-15
5,83a414de6ece18dd41a2e9f581b936a2,1,WT-15
6,05b31f3594fe22222610776222de7d2c,1,WT-15
7,30e6823a0bbf9c6c0db0dd35ed88a95e,1,WT-15
8,44f3e1ed02d4bf0ad9ffaaeec79529ef,1,WT-15
9,f745d339cc18edc5702d6ed5c1cfa00e,1,WT-15


In [49]:
teams = pd.merge(teaming, data, on='hash')[['team', 'Sex', 'Discipline', 'Nationality']]
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(teams)

    team Sex            Discipline             Nationality
0      0   f  Creative Disciplines  Dutch or Netherlandish
1      0   f            Humanities                  German
2      0   m              Business                  German
3      0   m           Engineering                  German
4      0   m                 Media                  German
5      1   f  Creative Disciplines                 Mexican
6      1   m           Engineering                  German
7      1   m           Engineering                 Italian
8      1   m              Business                  German
9      1   f         Life Sciences                  German
10     2   m         Life Sciences                  German
11     2   m           Engineering                  German
12     2   f              Business                   Cuban
13     2   f            Humanities                  German
14     2   m           Engineering                  German
15     3   f  Creative Disciplines               Brazili

In [25]:
len(teams[teams['Nationality'] != 'German'])

20

In [29]:
teams['Discipline'].value_counts()

Engineering             23
Business                18
Life Sciences           11
Humanities              10
Creative Disciplines     8
Social Sciences          8
Media                    3
Name: Discipline, dtype: int64

In [30]:
teams['Nationality'].value_counts()

German                    61
Dutch or Netherlandish     3
Italian                    3
Mexican                    2
Brazilian                  2
Sri Lankan                 1
Macedonian                 1
Chilean                    1
Indian                     1
Nepalese                   1
Japanese                   1
French                     1
Cuban                      1
American or US             1
Danish                     1
Name: Nationality, dtype: int64

In [31]:
teams['Sex'].value_counts()

m    46
f    35
Name: Sex, dtype: int64

In [118]:
def single_entropy(team):
    counter = Counter(team)
    l = len(team)
    return -sum((c/l) * math.log(c/l) for c in counter.values())
        
def entropy(teams):
    return sum(single_entropy(team) for team in teams)

def max_entropy(value_map):
    n_teams = 16
    teams = []
    for _ in range(n_teams):
        teams.append([])
    i = 0
    for attr_id, value_count in enumerate(value_map):
        base_n = value_count // n_teams
        for team in teams:
            team.extend([attr_id] * base_n)
        value_count = value_count % n_teams
        while value_count > 0:
            teams[i].append(attr_id)
            value_count -= 1
            i = (i + 1) % n_teams
    return entropy(teams)
    

def max_entropy_teaming(teaming):
    attrs = ['Sex', 'Nationality', 'Discipline']
    return sum(max_entropy(teaming[attr].value_counts()) for attr in attrs)

def print_entropies(teaming):
    attrs = ['Sex', 'Nationality', 'Discipline']
    for attr in attrs:
        print(attr)
        print('no teaming: {:0.4f}'.format(single_entropy(teams[attr].values)))
        print('teaming: {:0.4f}'.format(max_entropy(teaming[attr].value_counts())))
        print('')
    print('total teaming: {:0.4f}'.format(max_entropy_teaming(teaming)))


In [104]:
max_entropy(teams['Nationality'].value_counts())

9.723203928639418

In [106]:
max_entropy_teaming(teams)

43.99573423209929

In [108]:
single_entropy(teams['Nationality'].values)

1.182997695798113

In [119]:
print_entropies(teams)

Sex
no teaming: 0.6839
teaming: 10.7883

Nationality
no teaming: 1.1830
teaming: 9.7232

Discipline
no teaming: 1.8005
teaming: 23.4842

total teaming: 43.9957
